# Pronósticos de demanda diaria con LSTM usando Keras y TensorFlow

## Pronósticos a partir de la serie temporal de la demanda diaria incorporando variables exógenas desde Datasets

**Este es básicamente idéntico a StringLookup_5 salvo que se anticipa el preprocesamiento de los datos categóricos y se quita la cap `OneHotPre` del modelo formando el `Dataset` ya con las columnas tipo one-hot numéricas (antes al modelo llegaba un solo atributo pero del tipo `tf.string`**

La propuesta es dar pronósticos de la demanda diaria de energía eléctrica a partir de sus valores registrados en días anteriores y de otros atributos que podrían tener influencia (variables exógenas) alamcenados en `tf.data.Dataset`. 

Continuaremos con nuestro ejemplo tomando la base de datos de demanda diaria de energía eléctrica publicados por CAMMESA. 

## Inicio de la programación para el análisis y pronóstico de la serie temporal

Comenzamos importando las bibliotecas necesarias y confirmamos la versión de TensorFlow (TF) disponible en la instalación. Esto nos puede ayudar a interpretar algunas diferencias en el comportamiento de las funciones de TF.

In [ ]:
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

import keras
import tensorflow as tf
tf.random.set_seed(99)
print(tf.__version__)

## Carga del conjunto de datos para su análisis

Cargamos los datos de demanda diaria. DEMANDA TOTAL es la serie que queremos predecir (target). Esperamos que sus valores históricos nos provean información para su pronóstico. Además, tomaremos otros atributos que consideramos relevantes como `TEMPERATURA REFERENCIA MEDIA GBA °C` o `Tipo día` que nos informa si la jornada fue laborable, semilaborable o no laborable. Las columnas que se refieren a los registros de demanda para otras regiones argentinas no las consideraremos.

In [ ]:
# Dataset loading
dataFrame = pd.read_excel('Data/Base Demanda Diaria 2017 2024.xlsx', sheet_name='Datos Región', skiprows=4)  
dataFrame.info()

### Separación de los conjuntos de datos de entrenamiento, validación y prueba

Como es habitual, separamos el conjunto de datos en entrenamiento y prueba. Como sabemos, se reserva una porción del final de la serie temporal como datos de prueba. En este ejercicio, desdoblamos en datos numéricos y categóricos.

In [ ]:

# TensorFlow tensors require that all elements have the same dtype.
numeric_feature_names = ['DEMANDA TOTAL', 'TEMPERATURA REFERENCIA MEDIA GBA °C']
column_indices = {name: i for i, name in enumerate(numeric_feature_names)}
numeric_tensor = np.float32(dataFrame[numeric_feature_names].to_numpy())
# cast to tf defaults... float32

categoric_feature_names = ['Tipo día']
categoric_tensor = dataFrame[categoric_feature_names].to_numpy()



En lo siguiente utilizaremos `tf.keras.preprocessing.timeseries_dataset_from_array` que crea conjuntos de datos de ventanas móviles sobre series temporales provistas como arrays. Por esto mantenemos en el paso anterior la transformación `.to_numpy()`. 

## Alternativas para time series windows

usando `.batch()` y `.window()`

TF: Build TensorFlow input pipelines

[Time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing)

## Ventanas de datos




Necesitamos particionar la serie temporal en lotes operables por las sucesivas capas de NN. En este ejemplo tomaremos funciones desarrolladas específicamente para operar con series temporales en lugar de definir nuestras propias funciones que realizan los cortes en lotes de datos.

Como lo adelantamos, `tf.keras.preprocessing.timeseries_dataset_from_array` toma una secuencia de datos reunidos en intervalos iguales, junto con parámetros de la serie temporal como longitud de la secuencia/ventana, espaciamiento de la secuencia/ventana, etc., para producir lotes de series temporales de datos de entrada y de de targets.


In [ ]:

def make_dataset(data, target_idx=0, out_len=28, ts_num=28):
    input_data = data[:-ts_num]
    if target_idx is not None:
        targets = data[ts_num:, target_idx]
        targets = targets[...,np.newaxis]
    else:
        targets = None
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=input_data,
        targets=targets,
        sequence_length=out_len, #Length of the output sequences (in number of timesteps). 
        sequence_stride=1, #Period between successive output sequences
        shuffle=False,
        batch_size=ts_num, #Number of timeseries samples in each batch
        seed=43)
    return ds


In [ ]:
BATCH_SIZE=28
SEQ_LENGTH=28
num_attr_ds = make_dataset(numeric_tensor, 0, SEQ_LENGTH, BATCH_SIZE)
num_attr_ds.element_spec

Creamos un `Dataset` con los datos categóricos

In [ ]:
cattr_ds = make_dataset(categoric_tensor, None, SEQ_LENGTH, BATCH_SIZE)
cattr_ds.element_spec

In [ ]:
cattr_ds = cattr_ds.map(lambda X: tf.strings.lower(X))
cattr_ds = cattr_ds.map(lambda X: tf.strings.regex_replace(X, '^.*h.bil.*$', 'laborable'))
cattr_ds = cattr_ds.map(lambda X: tf.strings.regex_replace(X, '^.*semilaborable.*$', 'semilaborable'))
cattr_ds = cattr_ds.map(lambda X: tf.strings.regex_replace(X, '^.*feriado.*$', 'nolaborable'))

vocabulary = ['laborable', 'semilaborable', 'nolaborable']
one_hot_layer = tf.keras.layers.StringLookup(max_tokens = 3,
                                            num_oov_indices = 0,
                                            vocabulary = vocabulary,
                                            output_mode = 'one_hot')
one_hot_layer.get_config()

In [ ]:
cattr_ds = cattr_ds.map(lambda X: tf.cast(one_hot_layer(X), tf.float32))

for batch in cattr_ds.take(1):
    print(batch.shape)
    print(batch.dtype)



In [ ]:
nattr_ds = num_attr_ds.map(lambda X, y: X)
tgt_ds = num_attr_ds.map(lambda X, y: y)

#dict_ds = tf.data.Dataset.from_tensor_slices(({"num_input": nattr_ds}, {"output": tgt_ds},))
#dict_ds = tf.data.Dataset.from_tensor_slices((nattr_ds, tgt_ds))
# dict_ds = tf.data.Dataset.zip((nattr_ds, tgt_ds))
dict_ds = tf.data.Dataset.zip({'num_input': nattr_ds, 'cat_input': cattr_ds}, {'output': tgt_ds})
DATASET_SIZE = dict_ds.cardinality().numpy()
print(DATASET_SIZE)
dict_ds.element_spec



In [ ]:
train_size = int(0.82 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)

train_ds = dict_ds.take(train_size)
val_ds = dict_ds.skip(train_size).take(val_size)
test_ds = dict_ds.skip(train_size).skip(val_size)



In [ ]:
for batch in train_ds.take(5):
    inputs, outputs = batch
    # outputs = outputs[...,tf.newaxis]
    print(inputs.keys(), outputs.keys())

print('Input: ', inputs['num_input'].shape)
print('Input: ', inputs['cat_input'].shape)
print('Output: ', outputs['output'].shape)



Confirmamos los tamaños de los nuevos conjuntos de datos particionados en lotes.

Ahora no es necesario separar en atributos (`X`) y targets (`y`), solo debemos pasar los `Datasets` de entrenamiento o prueba

### Adaptar para normalizar los atributos numéricos 

In [ ]:
# Normalize numeric features
normalize_layer = keras.layers.Normalization(axis=2)
normalize_layer.adapt(train_ds.map(lambda X, y: X['num_input']))
normalize_layer.get_config()

Podemos conocer la media y la varianza que luego podríamos usar para invertir los resultados

In [ ]:
print('DEMANDA TOTAL - Mean: {0:.0f}, Std.Dev {1:.0f}'.format(normalize_layer.mean[0,0,0],
                                                           np.sqrt(normalize_layer.variance[0,0,0])))
print('Temperatura - Mean: {0:.1f}, Std.Dev. {1:.1f}'.format(normalize_layer.mean[0,0,1],
                                                             np.sqrt(normalize_layer.variance[0,0,1])))

## Definición del modelo 



En este caso, definimos el modelo de manera que procese la información de entrada por 2 canales diferentes. Los datos numéricos se normalizan, los categóricos pasan por la capa custom de codificación one-hot. Luego, ambas salidas se unen para continuar su procesamiento en la primer capa con celdas LSTM. 


## Como pasar multiples datos de entrada desde un Dataset

TF: Training & evaluation with the built-in methods 

the Dataset should return a tuple of dicts.

[Passing data to multi-input, multi-output models](https://www.tensorflow.org/guide/keras/training_with_built_in_methods#passing_data_to_multi-input_multi-output_models)

y también ver TF: The Functional API

When calling fit with a Dataset object, it should yield either a tuple of lists like `([title_data, body_data, tags_data], [priority_targets, dept_targets])` or a tuple of dictionaries like `({'title': title_data, 'body': body_data, 'tags': tags_data}, {'priority': priority_targets, 'department': dept_targets})`.

[Manipulate complex graph topologies](https://www.tensorflow.org/guide/keras/functional_api#manipulate_complex_graph_topologies)

In [ ]:
unnormalize_layer = keras.layers.Normalization(axis=1, mean=normalize_layer.mean[0,0,0],
                                              variance=normalize_layer.variance[0,0,0],
                                               invert=True, name='output')

# Functional API Model 
numeric_input = keras.layers.Input(shape=(SEQ_LENGTH, 2), dtype=tf.float32, name='num_input')
normalized = normalize_layer(numeric_input)
categorical_input = keras.layers.Input(shape=(SEQ_LENGTH, 3), dtype=tf.float32, name='cat_input')
concat = keras.layers.concatenate([normalized, categorical_input])
lstm_lyr = keras.layers.LSTM(10)(concat)
dropout_lyr = keras.layers.Dropout(0.2)(lstm_lyr)
dense_lyr = keras.layers.Dense(1, activation='linear')(dropout_lyr)
unnormalized = unnormalize_layer(dense_lyr)
model_exo = keras.models.Model(inputs=[numeric_input, categorical_input], outputs=[unnormalized])
model_exo.compile(loss = 'MeanSquaredError', metrics=['MAE'], optimizer='Adam')
#model_exo.compile(loss = {'output': tf.keras.losses.MeanSquaredError()},
#                  metrics={'output': tf.keras.metrics.MeanAbsoluteError()},
#                  optimizer='Adam')
model_exo.summary()

El método `.summary()` nos muestra cómo fluirá la información entre las capas de NN y cuántos parámetros se deberán ajustar en cada una. Otra alternativa para representar la configuración de la NN es usar `tf.keras.utils.plot_model`.

In [ ]:
tf.keras.utils.plot_model(model_exo, rankdir="LR", show_shapes=True,  show_layer_names=True)

### Ajuste del modelo

Entrenamos el modelo, es decir, ajustamos los pesos de acuerdo al esquema definido cuando fue compilado, por ejemplo optimizer='Adam'

In [ ]:

# Model training
history = model_exo.fit(train_ds, epochs=30, validation_data=val_ds)

# early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# history = model_exo.fit(train_ds, epochs=3000, validation_data=val_ds, callbacks=[early_stopping_cb])

Para tener una idea sobre la convergencia del moelo, mostramos las curvas de aprendizaje

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.yscale('log')
plt.grid(True)
# plt.gca().set_ylim(0,1)

Luego que el modelo ha sido ajustado, es muy fácil obtener nuevas predicciones, como por ejemplo cuando le pasamos el conjunto de prueba.

In [ ]:

# Predict discards targets
predicted_values = model_exo.predict(test_ds)

In [ ]:
predicted_values.shape

In [ ]:
#target_values = test_ds.map(lambda X, y: y['output'])
#target_values = target_values.flat_map(tf.data.Dataset.from_tensor_slices)
#print(type(target_values))
#target_values = np.fromiter(target_values.as_numpy_iterator(), dtype=np.float32)
#print(target_values.shape)

# or
target_values = test_ds.map(lambda X, y: y['output'])
target_values = target_values.unbatch()
print(type(target_values))
target_values = list(target_values.as_numpy_iterator())
print(len(target_values))

### Evaluación de los resultados

Ahora vamos a representar nuestro resultado. Construimos un `DataFrame` para almacenar los valores de `DEMANDA TOTAL` predecida y observada, y su fecha (que recuperamos de los datos originales). Este `DataFrame` nos facilitará el cálculo de distintas métricas que califican el grado de acuerdo de las predicciones del modelo con los valores registrados. Asimismo, representaremos en una figura ambas series de datos para tener una estimación visual del desempeño del modelo.  

In [ ]:
# Forecast Plot with Dates on X-axis
LSTM_eval = pd.DataFrame({
    'Predicted_DEMANDA': predicted_values[:, 0],
    'Actual_DEMANDA': target_values,
})

LSTM_eval.loc[:, 'Date'] = dataFrame['Fecha'][-len(target_values):].values
# LSTM_eval.set_index('Date', inplace=True)
LSTM_eval['Date'] = pd.to_datetime(LSTM_eval['Date'])
LSTM_eval.set_index('Date', inplace=True)


In [ ]:
# Creamos una figura para representar las series de datos
def FigPredActual(d, title):
    fig, ax = plt.subplots(figsize=(10, 6))
    #  highlight the  forecast
    # highlight_start = int(len(d) * 0.9)  
    # highlight_end = len(d) - 1  # Adjusted to stay within bounds
    # Plot the actual values
    # plt.plot(d[['Actual_DEMANDA']][:highlight_start], label=['Actual_DEMANDA'])
    plt.plot(d[['Actual_DEMANDA']], label=['Actual_DEMANDA'])
    
    # Plot predicted values with a dashed line
    plt.plot(d[['Predicted_DEMANDA']], label=['Predicted_DEMANDA'], linestyle='--')
    
    # Highlight the forecasted portion with a different color
    # plt.axvspan(d.index[highlight_start], d.index[highlight_end], facecolor='lightgreen', alpha=0.5, label='Forecast')
    
    plt.title(title)
    plt.xlabel('Dates')
    plt.ylabel('Values')
    ax.legend()

Trazamos ambas series de datos

In [ ]:
FigPredActual(LSTM_eval, 'Multivariate Time-Series forecasting using LSTM and Exogenous variables')

Definimos una función conveniente para calcular diferentes métrias que nos informan sobre el grado de acuerdo de las predicciones del modelo con las observaciones

In [ ]:
# Model Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def performance(d):
    return {
        'MSE': mean_squared_error(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy()),
        'MAE': mean_absolute_error(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy()),
        'R2': r2_score(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy())
    }

performance(LSTM_eval)

**Preguntas**

1. ¿Por qué el ajuste mejora si `BATCH_SIZE` y `SEQ_LENGTH` coinciden (sin importar demasiado si el número de días corresponde a un número entero de semanas?

**Ejercicio**

1. 